# Correlation Examples
These examples are based on pseudorandom sequences.  For the purposes of the notes, a particular sequence which has a high autocorrelation peak, an m-sequence, is used.

### Import function

The code to generate the m-sequence is long, and not directly related to correlation.  So, this is held in an external file, mseq_function.py.  To avoid the code being displayed as an output, in a Jupyter notebook, or ipython script, it is loaded with <code>%run</code>.  For standard Python, import is used.

In [ ]:
try:
    # in jupyter or ipynb system
    %run mseq_function.py
except: 
    # When download this jupyter notebook as .py file,
    # the line above will raise an error,
    # so in python system
    from mseq_function import *

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### Plotting function
To simplify the code, a function to produce the plots is defined below:

In [ ]:
def plot_figure(y, xlabel, ylabel, xlim, ylim, title, name):
    """
       Function that create a stem plot and 
       Save the figure with given name when not running in Jupyter notebook
       
       Parameters:
                  y (array-like): The y-values of the stem heads.
                  xlabel (str): The label for the x-axis.
                  ylabel (str): The label for the y-axis.
                  xlim ([left, right]): Set the xlim to left, right.
                  ylim ([left, right]): Set the ylim to left, right.
                  title (str): The title of figure.
                  name (str): Name used to save figure.
    """
    # Create the plot figure
    plt.figure(figsize = (12, 6))
    
    # Enlarge figure label and axis size
    plt.rcParams.update({'font.size': 16})
    
    # Generate the x-axis
    x = np.arange(xlim[0],xlim[1]+1)
    
    # stem plot and its marker style
    (markerLines, stemLines, baseLines) = plt.stem(x, y, use_line_collection = True)
    markerLines.set_markersize(4)
    plt.setp(stemLines, linewidth=1) 
    plt.setp(baseLines, color = 'black', linewidth=1) 
    
    # Tidy up the plot to control axes sizes and labels
    plt.xlim(xlim)    
    plt.ylim(ylim)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    
    # Save figure in python or ipython system
    if not is_jupyter(): plt.savefig(name)

### Define the input
Generate the input sequence as an m-sequence, which has good correlation properties.

In [ ]:
sequence = mseq(2,7)

Plot the sequence values

In [ ]:
# Plot the sequence to show its structure

plot_figure(y=sequence, xlim=[0, 126], ylim=[-4.2, 4.2],
            xlabel='Sample', ylabel='Amplitude',
            title='Input data sequence',
            name='correlation_input_data_sequence.pdf')

### Autocorrelation
The autocorrelation of the m-sequence is computed by correlating the sequence with itself.  The largest value occurs at zero displacement (delay), and for the m-sequence, all of the other values are small.

In [ ]:
# Compute the autocorrelation of the sequence 
# using the crosscorrelation function
rxx = np.correlate(sequence, sequence, 'full')

# Plot the result
plot_figure(y=rxx, xlim=[-126, 126], ylim=[-20, 140],
            xlabel='Delay', ylabel='Correlation',
            title='Input autocorrelation sequence',
            name='input_autocorrelation.pdf')

### Finding a signal in noise
One of the applications of correlation is to find a signal buried within noise.  Here we generate a noise sequence, add the m-sequence in, with a delay, and plot it.  From the plot it is not possible to identify that the signal is present.

In [ ]:
# Create a noisy, time-delayed received signal from the input sequence.
# In this case, SNR=0dB.  Start by generating the noise using
# numpy.random.randn(d0) which returns samples from the “standard normal”
# distribution.
corrupted = np.random.randn(127)

# Now add the signal in
corrupted[7:127] = corrupted[7:127] + sequence[0:120]

# Plot the noisy "received" signal
plot_figure(y=corrupted, xlim=[0, 126], ylim=[-4.2, 4.2],
            xlabel='Sample', ylabel='Amplitude',
            title='Received data sequence',
            name='correlation_received_data_sequence.pdf')

However, when the crosscorrelation of the original sequence and the noisy one is calculated, the presence of the sequence is obvious, as is the delay with which it has been added.  If you wish to convince yourself that this is true, replace <code>corrupted</code> below with <code>np.random.randn(127)</code> and rerun the cell.

In [ ]:
# Compute the crosscorrelation between the transmitted and received signals
rxy = np.correlate(sequence, corrupted, 'full')

# Plot the result
plot_figure(y=rxy, xlim=[-126, 126], ylim=[-40, 120],
            xlabel='Delay', ylabel='Correlation',
            title='Crosscorrelation result',
            name='crosscorrelation_result.pdf')

### Identifying periodicity

It is possible, without knowing the transmitted signal, to identify that a periodically repeating signal is present, even if it is corrupted by noise.  In the example below, the sequence above is repeated 79 times.  Noise is added with a power of 0 dB (as above), making it impossible to identify the signal from the time domain.  The autocorrelation of a periodic sequence is periodic, while the autocorrelation of the noise has a large value at 0 delay, but all other values are small.

In the plot below, the periodic repetitions can be identified from the large values at 127 and 254 (as well as -127 and -254).  The sequence has a length of 127, so this is the period of the repeating signal.  The larger value at the delay of 0 is a result of the noise autocorrelation added to the autocorrelation of the sequence at delay 0.

In [ ]:
# Generate a received signal from a periodic transmission of 
# the radar waveform
corrupted_periodic = np.random.randn(10000)

# Use repamt function to repeat sequence 79 times
periodic_stream = np.tile(sequence, 79)
corrupted_periodic[7:10000] = corrupted_periodic[7:10000] + \
                              periodic_stream[0:9993]

# Compute the autocorrelation of the received signal
ryy = np.correlate(corrupted_periodic, corrupted_periodic, mode='full')

# Plot only the central section of ryy, so that the detail can be seen
plot_figure(y=ryy[9699:10300],
            xlim=[-300,300], ylim=[-2000,21000],
            xlabel='Delay', ylabel='Correlation',
            title='Autocorrelation of received signal',
            name='receiver_autocorrelation_result.pdf')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.